Importing libries

In [95]:
import pandas as pd
import numpy as np

Uploading the FULL dataframe

In [96]:
dfFull = pd.read_excel("Faturamento_Truck.xlsx", sheet_name="FULL", header=3)

In [97]:
dfFull

,Unnamed: 0,NS,EMPRESA,ENDEREÇO,SITE,UF,PAIS,CEP,FABRICA,CC,...,MODELO,NS.1,IP,SSB - FILA,TP IMPR,CONT INIC,CONT FINAL,PAG IMPR,VALOR PAG,VALOR TOTAL
0,NaN,5.024866e+12,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-000,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.0,IMPRESSORATRUCK,Color,22788.0,22995.0,207,0.19,39.3300
1,NaN,5.024866e+12,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-001,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.1,IMPRESSORATRUCK,Color,19188.0,19362.0,174,0.19,33.0600
2,NaN,5.024866e+12,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-002,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.2,IMPRESSORATRUCK,Color,25992.0,26331.0,339,0.19,64.4100
3,NaN,5.024866e+12,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-003,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.3,IMPRESSORATRUCK,Color,14386.0,14386.0,0,0.19,0.0000
4,NaN,5.024866e+12,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-004,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.4,IMPRESSORATRUCK,Color,6443.0,6925.0,482,0.19,91.5800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,NaN,NaN,TOTAL EM BILHETAGEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,735651,NaN,24758.8904
220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUSTO FIXO,NaN,10000.0000
221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORÇAMENTOS,NaN,0.0000
222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TONER REC,NaN,7000.0000


Uploading the TONER REC dataframe

In [98]:
dfTonerRec = pd.read_excel("Faturamento_Truck.xlsx", sheet_name="TONER REC", header=6)

Uploading the Valor Fixo

In [99]:
dfValorFixo = pd.read_excel("Faturamento_Truck.xlsx", sheet_name="RESUMO - FATURAMENTO")

Delete columns with NaN columns in all cells

In [100]:
colunDel = dfFull.dropna(axis=1, how="all")

Delete lines with more than 5 NaN values

In [101]:
lineDel = colunDel.dropna(axis=0, thresh=5)

Replace NaN values with the last filled value in the line before

In [102]:
rplNullValue = lineDel.replace(np.nan, method="ffill")

Transforming column CC as int type

In [103]:
rplNullValue["CC"] = rplNullValue["CC"].astype("int")

Selecting columns with the valuable information

In [104]:
selectColumns = rplNullValue[["CC", "VALOR TOTAL"]]

Removing 0.0 values using a filter

In [105]:
filter = selectColumns["VALOR TOTAL"] != 0
selectColumns = selectColumns[filter]

Getting the fixed value to divide to all CC's

In [106]:
valorFixo = dfValorFixo.iat[6,5]

Getting the Full dataframe lenth

In [107]:
sizeFullDf = len(selectColumns)

Dividing the fixed value to all CC equally

In [108]:
rateioValorFixo = valorFixo / sizeFullDf

Adding fixed value into the dataframe full

In [109]:
selectColumns["RateioFixo"] = rateioValorFixo

Calculating the sum of total value with fixed value

In [110]:
selectColumns["TotalRateio"] = selectColumns["VALOR TOTAL"] + selectColumns["RateioFixo"]

Grouping dataframe full by CC

In [111]:
rateioAgrupado = selectColumns.groupby("CC").sum().reset_index()

Separating dataframe full by CC & TotalRateio

In [112]:
ccTotalRateio = rateioAgrupado[["CC", "TotalRateio"]]

Rounding column TotalRateio to 2 decimals

In [113]:
rateioFinal = ccTotalRateio.round({'TotalRateio': 2})

In [114]:
rateioFinal

,CC,TotalRateio
0,11111,9621.86
1,48899,820.93
2,55555,2236.68
3,56489,3984.68
4,66666,1100.11
5,88888,2164.82
6,97788,4267.24
7,98756,10232.38
8,99999,330.19


In [115]:
rateioFinal.to_csv("RateioFinal08.csv", header=False, index=False, sep=" ", float_format="%.2f")

In [116]:
np.sum(rateioFinal)

CC             624151.00
TotalRateio     34758.89
dtype: float64

------------------------------------------------------------------------------------------------------

Initializing Toner Rec analysis

In [117]:
dfTonerRec

,Supplier ID,Lexmark Customer Number,Lexmark Product Code,Product Part Desc,Quantity Shipped,Ship Date,Printer Serial Number,Order Number,Order Create Date,FABRICA,SITE,C/C,Valor
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-08 17:15:21,5.847889e+10,44477779.0,2021-01-01 12:57:18,NaN,NaN,55548.0,49.52625
2,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-09 00:00:00,5.847889e+10,44477779.0,2021-01-02 00:00:00,NaN,NaN,55548.0,49.52625
3,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-10 00:00:00,5.847889e+10,44477779.0,2021-01-03 00:00:00,NaN,NaN,55548.0,49.52625
4,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-11 00:00:00,5.847889e+10,44477779.0,2021-01-04 00:00:00,NaN,NaN,55548.0,49.52625
5,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-12 00:00:00,5.847889e+10,44477779.0,2021-01-05 00:00:00,NaN,NaN,55548.0,49.52625
6,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-13 00:00:00,5.847889e+10,44477779.0,2021-01-06 00:00:00,NaN,NaN,55548.0,49.52625
7,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-14 00:00:00,5.847889e+10,44477779.0,2021-01-07 00:00:00,NaN,NaN,55548.0,49.52625
8,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-15 00:00:00,5.847889e+10,44477779.0,2021-01-08 00:00:00,NaN,NaN,55548.0,49.52625
9,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-16 00:00:00,5.847889e+10,44477779.0,2021-01-09 00:00:00,NaN,NaN,55548.0,49.52625


Deleting lines with more than 5 NaN values

In [118]:
lineDelRec = dfTonerRec.dropna(axis=0, thresh=5)

In [119]:
lineDelRec

,Supplier ID,Lexmark Customer Number,Lexmark Product Code,Product Part Desc,Quantity Shipped,Ship Date,Printer Serial Number,Order Number,Order Create Date,FABRICA,SITE,C/C,Valor
1,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-08 17:15:21,5.847889e+10,44477779.0,2021-01-01 12:57:18,NaN,NaN,55548.0,49.52625
2,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-09 00:00:00,5.847889e+10,44477779.0,2021-01-02 00:00:00,NaN,NaN,55548.0,49.52625
3,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-10 00:00:00,5.847889e+10,44477779.0,2021-01-03 00:00:00,NaN,NaN,55548.0,49.52625
4,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-11 00:00:00,5.847889e+10,44477779.0,2021-01-04 00:00:00,NaN,NaN,55548.0,49.52625
5,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-12 00:00:00,5.847889e+10,44477779.0,2021-01-05 00:00:00,NaN,NaN,55548.0,49.52625
6,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-13 00:00:00,5.847889e+10,44477779.0,2021-01-06 00:00:00,NaN,NaN,55548.0,49.52625
7,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-14 00:00:00,5.847889e+10,44477779.0,2021-01-07 00:00:00,NaN,NaN,55548.0,49.52625
8,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-15 00:00:00,5.847889e+10,44477779.0,2021-01-08 00:00:00,NaN,NaN,55548.0,49.52625
9,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-16 00:00:00,5.847889e+10,44477779.0,2021-01-09 00:00:00,NaN,NaN,55548.0,49.52625
10,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-17 00:00:00,5.847889e+10,44477779.0,2021-01-10 00:00:00,NaN,NaN,55548.0,49.52625


Changind column C/C type to int

In [120]:
lineDelRec['C/C']  = lineDelRec['C/C'].astype('int')

<ipython-input-120-7fab81dcf94c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lineDelRec['C/C']  = lineDelRec['C/C'].astype('int')


Here I am verifying all CC with 0 values, getting the SITE column value and replacing the 0 by a not null SITE CC.

In [121]:
for linha in range(len(lineDelRec)):
    if lineDelRec['C/C'].iloc[linha] in [0]:
        x = lineDelRec['SITE'].iloc[linha]
        for linha2 in range(len(lineDelRec)):
            if lineDelRec['SITE'].iloc[linha2] in [x] and lineDelRec['C/C'].iloc[linha2] != 0 :
                y = lineDelRec['C/C'].iloc[linha2]
                lineDelRec['C/C'].iloc[linha] = y

In [122]:
lineDelRec

,Supplier ID,Lexmark Customer Number,Lexmark Product Code,Product Part Desc,Quantity Shipped,Ship Date,Printer Serial Number,Order Number,Order Create Date,FABRICA,SITE,C/C,Valor
1,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-08 17:15:21,5.847889e+10,44477779.0,2021-01-01 12:57:18,NaN,NaN,55548,49.52625
2,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-09 00:00:00,5.847889e+10,44477779.0,2021-01-02 00:00:00,NaN,NaN,55548,49.52625
3,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-10 00:00:00,5.847889e+10,44477779.0,2021-01-03 00:00:00,NaN,NaN,55548,49.52625
4,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-11 00:00:00,5.847889e+10,44477779.0,2021-01-04 00:00:00,NaN,NaN,55548,49.52625
5,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-12 00:00:00,5.847889e+10,44477779.0,2021-01-05 00:00:00,NaN,NaN,55548,49.52625
6,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-13 00:00:00,5.847889e+10,44477779.0,2021-01-06 00:00:00,NaN,NaN,55548,49.52625
7,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-14 00:00:00,5.847889e+10,44477779.0,2021-01-07 00:00:00,NaN,NaN,55548,49.52625
8,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-15 00:00:00,5.847889e+10,44477779.0,2021-01-08 00:00:00,NaN,NaN,55548,49.52625
9,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-16 00:00:00,5.847889e+10,44477779.0,2021-01-09 00:00:00,NaN,NaN,55548,49.52625
10,Printer Corporation,1.0,PRINTER,PRINTER FILE QUALITY,1.0,2021-09-17 00:00:00,5.847889e+10,44477779.0,2021-01-10 00:00:00,NaN,NaN,55548,49.52625


Selecting columns with the relevant information

In [130]:
lineDelRec = lineDelRec[['C/C', 'Valor']]
lineDelRec = lineDelRec.round({'Valor': 2})

In [131]:
lineDelRec

,C/C,Valor
0,25488,247.63
1,44558,495.26
2,55548,841.95
3,78895,445.74
4,88774,396.21
5,668984,346.68


Getting Fixed value from Toner Rec

In [132]:
valorFixoRec = dfValorFixo.iat[7,5]

Getting the full dataframe lenth

In [133]:
lineDelRec = lineDelRec.groupby('C/C').sum().reset_index()

In [134]:
lineDelRec

,C/C,Valor
0,25488,247.63
1,44558,495.26
2,55548,841.95
3,78895,445.74
4,88774,396.21
5,668984,346.68


In [128]:
np.sum(lineDelRec)

C/C      962247.00
Valor      2773.47
dtype: float64

In [129]:
lineDelRec.to_csv("RateioFinalRec.csv", header=False, index=False, sep=" ", float_format="%.2f")